In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import code_step2 as cs

# Load data

In [2]:
importlib.reload(cs)
num_profiles = 300
num_minutes = 60

# Generate 300 stochastic load profiles
all_profiles = np.array([cs.generate_profile(num_minutes=num_minutes) for _ in range(num_profiles)])
print(all_profiles.shape)

in_sample = all_profiles[0:100,:].copy()
out_sample = all_profiles[100:,:].copy()
print(in_sample.shape)
print(out_sample.shape)

(300, 60)
(100, 60)
(200, 60)


# Cvar

In [35]:
importlib.reload(cs)

eps=0.1
cvar_capacity, cvar_problem = cs.cvar(in_sample, eps, verbose=False)
print(f'Offered capacity for FCR-D UP: {cvar_capacity} kW')

Offered capacity for FCR-D UP: 220.06002224617973 kW


# ALSO-X

In [34]:
importlib.reload(cs)

eps=0.1
also_x_capacity, also_x_problem = cs.also_x(in_sample, eps, verbose=False)
print(f'Offered capacity for FCR-D UP: {also_x_capacity} kW')

Iteration: 0
0.0 0.3 0 0.3
Iteration: 1
0.0 0.15 0 0.15
Iteration: 2
0.0 0.075 0 0.075
Iteration: 3
0.0 0.0375 0 0.0375
Iteration: 4
0.0 0.01875 0 0.01875
Iteration: 5
0.0 0.009375 0 0.009375
Iteration: 6
0.0 0.0046875 0 0.0046875
Iteration: 7
0.0 0.00234375 0 0.00234375
Iteration: 8
0.0 0.001171875 0 0.001171875
Iteration: 9
0.0 0.0005859375 0 0.0005859375
Iteration: 10
0.0 0.00029296875 0 0.00029296875
Iteration: 11
0.0 0.000146484375 0 0.000146484375
Iteration: 12
0.0 7.32421875e-05 0 7.32421875e-05
Offered capacity for FCR-D UP: 220.0600222461817 kW


# Out of sample analysis

In [22]:
all_out_samples = out_sample.flatten()

np.quantile(all_out_samples, 0.1)

261.32990890273715